In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, pipeline, AutoModel
import re
import io
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score, accuracy_score,precision_score,recall_score
import os
from google.colab import drive
seed =45
np.random.seed(seed)
tf.random.set_seed(seed)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tf.distribute.cluster_resolver.TPUClusterResolver.connect())
drive.mount('/gdrive')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.116.24.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.116.24.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
model_type="UBC-NLP/MARBERT"
language="Ara"
data_dir="/content"
train_file="ara_train.csv"
dev_file="ara_dev.csv"
model_dir="/content"
model_file=f"{language}-Mono-Model.h5"
train_path=os.path.join(data_dir,train_file)
dev_path=os.path.join(data_dir,dev_file)
model_path=os.path.join(model_dir,model_file)

In [ ]:
train_data=pd.read_csv(train_path)
valid_data=pd.read_csv(dev_path)

In [ ]:
def balanced_batch(data):
  harmful=data.loc[data["class_label"]==1]
  norm_count=int(len(harmful)*1.5)
  normal=data.loc[data["class_label"]==0].sample(n=norm_count)
  data=harmful.append(normal).sample(frac=1).reset_index(drop=True)
  return data

train_data=balanced_batch(train_data)

In [ ]:
def gen_data(data,tokenizer_type=model_type,test_data=False):
    data_dict={}
    if not test_data:
      data_dict["harmful_label"]=tf.convert_to_tensor(data["class_label"].values.astype(int))
    data_dict["text"]=data["tweet_text"].values.astype("str")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
    input_ids, input_masks = [],[]
    for sentence in data_dict["text"]:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=326,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
    data_dict["input_ids"]=tf.convert_to_tensor(input_ids) 
    data_dict["input_masks"]=tf.convert_to_tensor(input_masks)  
    return data_dict

train=gen_data(train_data)
valid=gen_data(valid_data)

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def create_mono_model(transformer):
    input_ids= tf.keras.layers.Input(shape=(326,), dtype='int32')
    input_masks = tf.keras.layers.Input(shape=(326,), dtype='int32')
    embedding_layer =transformer(input_ids, attention_mask=input_masks)[0]
    #embedding_layer= tf.keras.layers.SpatialDropout1D(0.2)(embedding_layer)
    
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.5))(embedding_layer)
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.4))(harmful)
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, dropout=0.3))(harmful)
    harmful = tf.keras.layers.Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(harmful)
    harmful = tf.keras.layers.Dropout(0.2)(harmful)
    avg_pool_harmful = tf.keras.layers.GlobalAveragePooling1D()(harmful)
    max_pool_harmful = tf.keras.layers.GlobalMaxPooling1D()(harmful)
    harmful = tf.keras.layers.concatenate([avg_pool_harmful, max_pool_harmful])
    harmful = tf.keras.layers.Dropout(0.05)(harmful)
    output_harmful=tf.keras.layers.Dense(1, activation="sigmoid",name="output_harmful")(harmful)

    model = tf.keras.Model(inputs=[input_ids, input_masks], outputs =output_harmful)
    
    return model

In [ ]:
def build_mono_model(compile=False,show_summary=False,pretrained_model=model_type):
    with tpu_strategy.scope():
      config =AutoConfig.from_pretrained(pretrained_model,dropout=0.2,seed=3,attention_dropout=0.2,output_hidden_states = True)
      transformer= TFAutoModel.from_pretrained(pretrained_model,config=config)
      model =create_mono_model(transformer)
      if compile : 
          model.compile(loss={"output_harmful":"binary_crossentropy"},
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                      metrics={"output_harmful":"accuracy"}
                     )
      if show_summary:
        model.summary()
      return model

model=build_mono_model()

def train_model(model,train,valid,batch_size=64,epochs_frozen=4,epochs_unfrozen=12):
      
    # checkpoint_filepath = data_path

    # model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    #     checkpoint_filepath, 
    #     monitor='val_loss', 
    #     save_weights_only=True, 
    #     save_best_only=True, 
    #     save_freq='epoch', 
    #     options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'))
    
    for layer in model.layers[:3]:
        layer.trainable = False
   
    with tpu_strategy.scope():
      model.compile(loss={"output_harmful":"binary_crossentropy"},
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                      metrics={"output_harmful":"accuracy"}
                     )
        
    model.fit([train["input_ids"], train["input_masks"]],train["harmful_label"],batch_size=batch_size,epochs=epochs_frozen,validation_data=([valid["input_ids"],valid["input_masks"]],valid["harmful_label"]))#, callbacks=[model_checkpoint_callback])
   
    for layer in model.layers[:3]:
        layer.trainable = True

    with tpu_strategy.scope():
      model.compile(loss={"output_harmful":"binary_crossentropy"}, 
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                      metrics={"output_harmful":"accuracy"})
    
    model.fit([train["input_ids"], train["input_masks"]],train["harmful_label"],batch_size=batch_size,epochs=epochs_unfrozen,validation_data=([valid["input_ids"],valid["input_masks"]],valid["harmful_label"]))#, callbacks=[model_checkpoint_callback])

train_model(model,train,valid)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/MARBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/4
29/29 [==============================] - 101s 2s/step - loss: 0.6782 - accuracy: 0.5832 - val_loss: 0.4615 - val_accuracy: 0.8099
Epoch 2/4
29/29 [==============================] - 10s 350ms/step - loss: 0.6292 - accuracy: 0.6434 - val_loss: 0.4507 - val_accuracy: 0.8099
Epoch 3/4
29/29 [==============================] - 10s 348ms/step - loss: 0.5995 - accuracy: 0.6650 - val_loss: 0.4360 - val_accuracy: 0.8099
Epoch 4/4
29/29 [==============================] - 10s 348ms/step - loss: 0.5694 - accuracy: 0.7133 - val_loss: 0.4266 - val_accuracy: 0.8099
Epoch 1/12


29/29 [==============================] - 224s 4s/step - loss: 0.4296 - accuracy: 0.7864 - val_loss: 0.2560 - val_accuracy: 0.8843
Epoch 2/12
29/29 [==============================] - 17s 581ms/step - loss: 0.2769 - accuracy: 0.8856 - val_loss: 0.2418 - val_accuracy: 0.8913
Epoch 3/12
29/29 [==============================] - 17s 584ms/step - loss: 0.1689 - accuracy: 0.9442 - val_loss: 0.2909 - val_accuracy: 0.8813
Epoch 4/12
29/29 [==============================] - 17s 582ms/step - loss: 0.0967 - accuracy: 0.9707 - val_loss: 0.3303 - val_accuracy: 0.8773
Epoch 5/12
29/29 [==============================] - 17s 583ms/step - loss: 0.0546 - accuracy: 0.9827 - val_loss: 0.3552 - val_accuracy: 0.8843
Epoch 6/12
29/29 [==============================] - 17s 583ms/step - loss: 0.0369 - accuracy: 0.9919 - val_loss: 0.3964 - val_accuracy: 0.8803
Epoch 7/12
29/29 [==============================] - 17s 583ms/step - loss: 0.0287 - accuracy: 0.9919 - val_loss: 0.4331 - val_accuracy: 0.8763
Epoch 8/12
2

In [ ]:
def report_gen(predictions,labels,avg="binary"):
    report={
    "F1_macro":f1_score(predictions,labels,average=avg),
    "Accuracy":accuracy_score(predictions,labels),
    "Precision_macro":precision_score(predictions,labels),
    "Recall_macro":recall_score(predictions,labels)
    }
    return report

def eval_task_harmful(predictions_harmful,labels_harmful):
  predictions_harmful=[int(i>0.9) for i in predictions_harmful]
  return report_gen(predictions_harmful,labels_harmful),predictions_harmful

def eval_model(model,valid):
    predictions_harmful=model.predict([valid["input_ids"],valid["input_masks"]])
    report_harmful,predictions_harmful=eval_task_harmful(predictions_harmful,valid["harmful_label"])
    print(report_harmful)
    return predictions_harmful

predictions=eval_model(model,valid)

In [ ]:
def predict_test(test_data):
  test=gen_data(test_data,test_data=True)
  preds=model.predict([test["input_ids"],test["input_masks"]])
  preds=[int(i>0.9) for i in preds]
  data=pd.DataFrame({"topic":test_data["topic"],"tweet_id":test_data["tweet_id"],"class_label":preds,"run_id":"iCompass"})
  data.to_csv("subtask1C_claim_arabic.tsv",sep="\t",index=False)
predict_test(pd.read_csv("test.tsv",sep="\t"))


FileNotFoundError: ignored